In [1]:
# Install cirq
import sys
!pip install --upgrade pip
!{sys.executable} -m pip install cirq

Requirement already up-to-date: pip in c:\users\kevin\anaconda3\lib\site-packages (19.3.1)


In [4]:
# Import dependencies
import cirq
import numpy as np

In [3]:
# Check that cirq was installed and imported successfully
print(cirq.google.Foxtail)

(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)



This notebook implements the Bernstein-Vazirani Algorithm, which solves the following problem:

Given a $n$-bit string $s$, determine $s$ using only queries of the form $f(q) = q \cdot s % 2$  

To solve this classically would require $n$ queries to an oracle, but can accomplished quantumly with only a single oracle call. 

In [9]:
# Specify n and randomly generate a n-bit string.
n = 8
s = np.random.randint(0, 2, n)
s_value = 0
for i in range(n):
    s_value += 
print(s)

[0 1 1 0 1 1 1 0]


In [23]:
# Initialize our qubits and our scratch qubits
qubits = [cirq.GridQubit(i, 0) for i in range(n)]
scratch = cirq.GridQubit(n, 0)

print(qubits)
print(scratch)

[cirq.GridQubit(0, 0), cirq.GridQubit(1, 0), cirq.GridQubit(2, 0), cirq.GridQubit(3, 0), cirq.GridQubit(4, 0), cirq.GridQubit(5, 0), cirq.GridQubit(6, 0), cirq.GridQubit(7, 0)]
(8, 0)


In [28]:
# Start off the circuit by passing in a uniform superposition and a |1>
circuit = cirq.Circuit()

for i in range(n):
    circuit.append(cirq.H(qubits[i]))
    
circuit.append(cirq.X(scratch))
circuit.append(cirq.H(scratch))

print(circuit)

(0, 0): ───H───────

(1, 0): ───H───────

(2, 0): ───H───────

(3, 0): ───H───────

(4, 0): ───H───────

(5, 0): ───H───────

(6, 0): ───H───────

(7, 0): ───H───────

(8, 0): ───X───H───


In [29]:
# Implement the oracle as a series of CNOT gates
for i in range(n):
    if s[i] == 1:
        circuit.append(cirq.CNOT.on(qubits[i], scratch))
        
print(circuit)

(0, 0): ───H───────────────────────────

(1, 0): ───H───────@───────────────────
                   │
(2, 0): ───H───────┼───@───────────────
                   │   │
(3, 0): ───H───────┼───┼───────────────
                   │   │
(4, 0): ───H───────┼───┼───@───────────
                   │   │   │
(5, 0): ───H───────┼───┼───┼───@───────
                   │   │   │   │
(6, 0): ───H───────┼───┼───┼───┼───@───
                   │   │   │   │   │
(7, 0): ───H───────┼───┼───┼───┼───┼───
                   │   │   │   │   │
(8, 0): ───X───H───X───X───X───X───X───


In [30]:
# Undo the Hadamards and measure
for i in range(n):
    circuit.append(cirq.H(qubits[i]))

circuit.append(cirq.measure(*qubits, key='x'))

print(circuit)

(0, 0): ───H───H───────────────────────────M('x')───
                                           │
(1, 0): ───H───────@───H───────────────────M────────
                   │                       │
(2, 0): ───H───────┼───@───H───────────────M────────
                   │   │                   │
(3, 0): ───H───H───┼───┼───────────────────M────────
                   │   │                   │
(4, 0): ───H───────┼───┼───@───H───────────M────────
                   │   │   │               │
(5, 0): ───H───────┼───┼───┼───@───H───────M────────
                   │   │   │   │           │
(6, 0): ───H───────┼───┼───┼───┼───@───H───M────────
                   │   │   │   │   │       │
(7, 0): ───H───H───┼───┼───┼───┼───┼───────M────────
                   │   │   │   │   │
(8, 0): ───X───H───X───X───X───X───X────────────────


In [31]:
# Run the circuit and see what we get!
simulator = cirq.Simulator()
results = simulator.run(circuit, repetitions=100)

print("Expected: ", s)
print("Actual:", results.histogram(key='x'))

Expected:  [0 1 1 0 1 1 1 0]
Actual: Counter({110: 100})
